## Header

### Overview

Breast cancer ranks first for incidence in females in 159 countries, and it alone accounted for approximately 30% of new female cancers in U.S. in 2020. The World Health Organization and American Cancer Association recommend organized and population-wide breast cancer screening to promote early detection and reduce mortality. Mammography has been the primary method for breast cancer screening, and the overall sensitivity of mammography for non-palpable cancers detection is about 85%. But the sensitivity of mammography detection of breast cancer for extremely dense breasts can be as low as 61%. In American women, it was estimated that 27.6 million women aged 40-74 years have heterogeneously or extremely dense breasts. Many studies has suggested that using breast ultrasound as supplemental screening with mammography or as primary imaging could significantly improve the sensitivity of cancer detection in women with dense breast.

 
Recent research demonstrated that BUS with advanced machine learning approaches could achieve high sensitivity (94%-95%) and high specificity (89%-93%) for breast cancer detection. However, most approaches were trained and evaluated using small private datasets, and their performances degrade dramatically on datasets from different sources. This major drawback makes it difficult to deploy and adopt them in an open-world environment where we may encounter data with large variations.

In this competition, you will develop approaches for breast cancer detection using a large B-model breast ultrasound dataset. More specifically, you will develop an approach that can automatically classify breast ultrasound images into the ‘benign’ and ‘malignant’ categories.

 

### Dataset

The dataset was collected from five countries and contains a total of 3,600 B-model breast ultrasound images, of which 1,837 contain benign tumors, and 1,763 are malignant tumors.

Note the dataset can only be used for this competition, and sharing and usage for all other purposes are not allowed. Please sign the release agreement and send it to our TA. The dataset is available at http://bus.midalab.net/labeling Links to an external site.(Sign-up using your university email is required.)

Release agreement

### Evaluation

To evaluate the performance of the approach, we use the following quantitative metrics: accuracy (Acc), sensitivity (Sens), and specificity (Spec). Acc values will be used as the primary ranking score, and Sens will be used as a secondary ranking score if a tie happens.

 

 Acc = (TP+ TN)/(TP+FP+TN+FN)
 Sens = TP/(TP+FN)
 Spec = ( TN)/(TN+FP)
In Eqs.(1-3), TP is the number of true positives, TN is the number of true negatives, FP is the number of false positives, and FN is the number of false negatives.

 

### Submission 

1. Please use this code Download this codeto prepare your data and save your results to 'test_pred.csv'. The csv file should have a 'name' and a 'pred' column. Here is a sample csv file test_pred.csv Download test_pred.csv.

2. Submit the following files: 1) Python code, and 2) results (test_pred.csv) 

## References
https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/
https://www.tensorflow.org/tutorials/images/transfer_learning#fine_tuning 

## Colab

- train.csv containing image name and target required
- folders of test and training images required

<a href="https://colab.research.google.com/github/SethCram/CS474-Deep-Learning/blob/main/HW4/HW4_CNN_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Google Colab Cell

#enable debugging
!pip install -Uqq ipdb
import ipdb
%pdb on

#mount google drive
from google.colab import drive
drive.mount('/content/drive')

#seth act
%cd /content/drive/MyDrive/School/Senior Year/CS 474-01 (Deep Learning)/
#other act
%cd /content/drive/MyDrive/DL-Proj4/

#make sure GPU enabled
!nvidia-smi

#install proper TF version for convnext
!pip install tensorflow==2.10.0

## Imports

In [ ]:
from tensorflow.keras.applications import convnext
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import keras 
from keras import layers
from keras import optimizers
from keras import applications
from keras import losses
from keras import callbacks
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import cv2 as cv
import csv
from sklearn.utils import shuffle
import os

print("Modules imported")

# Data Import

In [ ]:
#LOAD CSV AND IMAGES (in proper resolution)

df=pd.read_csv('train.csv', sep = ',')
print(df)

training_imgs_path = 'trainImgs/'
test_imgs_path = 'testImgs/'

targetWidth = 512 #500 #256 for imagenet
targetHeight = 512 #500

# store training and test imgs greyscaled 
x_train = np.array([
            cv.resize( #resize to desired size
                cv.imread(training_imgs_path + row[1] + ".png"), 
                dsize=(targetWidth, targetHeight),
            ) 
            for row in df.values
    ])
x_test = np.array([
            cv.resize( #resize to desired size
                cv.imread(test_imgs_path + str(i) + ".png"),
                dsize=(targetWidth, targetHeight),
            )
            for i in range(1248)
    ])

#store whether benign (0?) or malignant (1?)
y_train = np.array([row[2] for row in df.values], dtype='float32')

nImg = 4  
for i in range(nImg*nImg):
    plt.subplot(nImg, nImg, i+1)
    plt.imshow(x_train[i])
plt.show()

In [ ]:
## ANOTHER WAY TO IMPORT DATA (possibly faster)

# load all training images and labels
train_img_folder = 'trainImgs'
train = pd.read_csv('train.csv')
print(train)

targetWidth = 512 
targetHeight = 512 
input_size = 512

trainImg_names = list(train['img name']) 
print('loading images ...')
x_train = []
for idx, name in enumerate(trainImg_names):
    img = image.load_img(os.path.join(train_img_folder, name +'.png'), target_size = (input_size, input_size)) #color_mode = "grayscale",
    img = image.img_to_array(img)
    x_train.append(img)
    #print(idx, name)
x_train = np.array(x_train).astype(np.float32)
y_train = np.array(train['tumor types'])

# convert class vectors to binary class matrices
#y_train_onehot = tf.keras.utils.to_categorical(y_train)
#print(y_train_onehot.shape)

print('loading finished ...')
#print(x_train.shape, y_train_onehot)

nImg = 4  
for i in range(nImg*nImg):
    plt.subplot(nImg, nImg, i+1)
    plt.imshow(x_train[i])
plt.show()

# Transfer Learning
## Model Creation

### Comon Model Layers

In [ ]:
input_layer = layers.Input(shape=(targetWidth, targetHeight, 3))

#data augmentation
flip_layer = layers.RandomFlip()
rot_layer = layers.RandomRotation( #seems to throw off models and slow down training
        0.2,
        fill_mode="constant",
        fill_value=0,
    )
translation_layer = layers.RandomTranslation(
    height_factor=(-0.2, 0.2),
    width_factor=(-0.2, 0.2),
    fill_mode="constant",
    fill_value=0,
)

output_layer = layers.Dense(
    units=1,
    activation=keras.activations.sigmoid 
)

nImg = 4  
for i in range(nImg*nImg):
    plt.subplot(nImg, nImg, i+1)
    
    aug_image = flip_layer(x_train[i])
    aug_image = translation_layer(aug_image)
    aug_image = rot_layer(aug_image)
    
    plt.imshow(aug_image.numpy().astype("uint8"))
plt.show()

### Common Model Blocks

In [ ]:
x = flip_layer(input_layer)
x = rot_layer(x)
x = translation_layer(x)

### VGG Model

In [ ]:
## VGG

filename = "vgg16_firstpass_retry_model"

#avg = 530
#avg_training_width = sum( image.shape[0] for image in x_train ) / x_train.shape[0]
#avg_training_height = sum( image.shape[1] for image in x_train ) / x_train.shape[0]
#avg_test_width = sum( image.shape[0] for image in x_test ) / x_test.shape[0]
#avg_test_height = sum( image.shape[1] for image in x_train ) / x_train.shape[0]

# layer construction
preprocc_input = applications.vgg16.preprocess_input
base_model = applications.VGG16(
    input_shape=(targetWidth, targetHeight, 3),
    include_top=False,
)
flatten_layer = layers.Flatten()
fc_layer = layers.Dense(512, activation='relu')
# Add a dropout rate of 0.5
dropout_layer = layers.Dropout(0.5)

base_model.trainable = False

#layer connecting
x = preprocc_input(x) #training should be true here?
x = base_model(x, training=False)
x = flatten_layer(x)
x = fc_layer(x)
x = dropout_layer(x)
predictions = output_layer(x)
model = keras.Model(input_layer, predictions)

### InceptionV3 Model

In [ ]:
filename = "inceptronv3_firstpass_retry_model"

# layer construction
preprocc_input = applications.inception_v3.preprocess_input

base_model = applications.InceptionV3(
    input_shape=(targetWidth, targetHeight, 3),
    include_top=False,
)
base_model.trainable = False

global_avg_pool_layer = layers.GlobalAveragePooling2D()
fc_layer = layers.Dense(1024, activation='relu')
dropout_layer = layers.Dropout(0.2)

#layer connecting
x = preprocc_input(x) #training should be true here?
x = base_model(x, training=False)
x = global_avg_pool_layer(x)
x = fc_layer(x)
x = dropout_layer(x)
predictions = output_layer(x)
model = keras.Model(input_layer, predictions)

### Efficient Net Model

In [ ]:
filename = "EffNetB2_firstpass_model"

# layer construction
preprocc_input = applications.efficientnet.preprocess_input
base_model = applications.EfficientNetB2(
    input_shape=(targetWidth, targetHeight, 3),
    include_top=False,
)
base_model.trainable = False

flatten_layer = layers.Flatten()
fc_layer = layers.Dense(512, activation='relu')
dropout_layer = layers.Dropout(0.3) 

#layer connecting
x = preprocc_input(x) #training should be true here?
x = base_model(x, training=False)
x = flatten_layer(x)
x = fc_layer(x)
x = dropout_layer(x)
predictions = output_layer(x)
model = keras.Model(input_layer, predictions)

### ConvNext Model

In [ ]:
filename = "ConvNextTiny_finetune_firstpass_model"

# layer construction
base_model = convnext.ConvNeXtTiny( #preproccing included
    input_shape=(targetWidth, targetHeight, 3),
    include_top=False,
)
base_model.trainable = False

flatten_layer = layers.Flatten()
#fc_layer = layers.Dense(216, activation='relu')
#dropout_layer = layers.Dropout(0.3) 

#layer connecting
x = base_model(x, training=False)
x = flatten_layer(x)
#x = fc_layer(x)
#x = dropout_layer(x)
predictions = output_layer(x)
model = keras.Model(input_layer, predictions)

### MobileNet V2 Model

In [ ]:
filename = "mobilenetv2_firstpass_finetuned3_model"

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
base_model = tf.keras.applications.MobileNetV2(
        input_shape=(targetWidth, targetHeight, 3),
        include_top=False,
        weights='imagenet'
    )
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

#layer connecting
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(input_layer, outputs)

### HuggingFace Model

In [ ]:
## UNTRAINABLE (doesn't use images)

from transformers import AutoTokenizer, TFAutoModel

filename = "ENM/sciBERT-case-finetuned-breastcancer"
num_classes = 1
batch_size = 32

tokenizer = AutoTokenizer.from_pretrained(filename)

model = TFAutoModel.from_pretrained(filename, from_pt=True, num_labels=num_classes)
#model.trainable = True

model.summary(expand_nested=True, show_trainable=True)

## Model Compilation

### Setup

In [ ]:
base_learning_rate = 0.0001
num_epochs = 10

num_train_steps = len(x_train) * num_epochs

lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=base_learning_rate, end_learning_rate=0.0,
        decay_steps=num_train_steps
    )

### Compilation

In [ ]:
model.compile(
    optimizer=optimizers.Adam(), #learning_rate=lr_scheduler
    loss=losses.BinaryCrossentropy(), #from_logits=True
    metrics=['accuracy']
)

In [ ]:
model.compile(
    optimizer=optimizers.RMSprop(), #optimizers.rmsprop(lr=0.0001, decay=1e-6) #sometimes decay not included?
    loss=losses.BinaryCrossentropy(), #from_logits=True
    metrics=['accuracy']
)

In [ ]:
#need to recompile model
model.compile(
    optimizer=optimizers.SGD(learning_rate=base_learning_rate, momentum=0.9), 
    loss=losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

### Post Compilation

In [ ]:
model.summary(expand_nested=True, show_trainable=True)

## Model Restoration

### Restore Model

In [ ]:
## RESTORE THE MODEL

filename = "ConvNextTiny_firstpass_model"

model = keras.models.load_model("saved_models/" + filename)

model.summary(expand_nested=True, show_trainable=True)

filename = "ConvNextTiny_firstpass2_model"

### Restore Weights

In [ ]:
filename = "inceptronv3_dropout3_finetuned_model"

model = keras.models.load_model("checkpoints/" + filename)

model.summary(expand_nested=True, show_trainable=True)

filename = "inceptronv3_dropout3_finetuned2.1_model"

### Fine Tuning

shouldn't be finetuning if additional dense layer before output layer?

In [ ]:
base_model_index = 5

#enable base model training
model.layers[base_model_index ].trainable = True

#disable all base model layers
for base_model_layer in model.layers[base_model_index ].layers:
    base_model_layer.trainable = False

#make some conv2D layers trainable
model.layers[base_model_index ].layers[299].trainable = True

model.layers[base_model_index ].layers[294].trainable = True

model.layers[base_model_index].layers[292].trainable = True
model.layers[base_model_index].layers[291].trainable = True
model.layers[base_model_index].layers[290].trainable = True
model.layers[base_model_index].layers[289].trainable = True

model.layers[base_model_index].layers[284].trainable = True
model.layers[base_model_index].layers[283].trainable = True


model.summary(expand_nested=True, show_trainable=True)

In [ ]:
model.layers[5].trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 50

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[5].layers[:fine_tune_at]:
  layer.trainable = False
  
model.summary(expand_nested=True, show_trainable=True)

## Model Training

### Setup

In [ ]:
#shuffle data everytime train
x_train, y_train = shuffle(x_train, y_train)

batch_size = 32
num_epochs = 10

early_stopping_callback = callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=4,
    verbose=1,
    restore_best_weights=True
)

logger_callback = callbacks.CSVLogger(
    filename="model_logs/" + filename + '.csv',
    separator=',',
    append=True #don't overwrite if file exists already
)

#for periodically saving best model
checkpoint_path = 'checkpoints/' + filename
checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True
)

tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir='tb_logs',
    histogram_freq=2,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq='epoch',
)

### Fitting

In [ ]:
#if retrain same model again:
# filename = "inceptronv3_firstpass_retry_finetuned2_model"

history = model.fit(
    x_train, 
    y_train, 
    epochs=num_epochs, 
    batch_size=batch_size,  #32 works well
    validation_split=0.2, 
    validation_batch_size=batch_size,
    shuffle=True, #shuffles data after every epoch to reduce overfitting
    callbacks=[logger_callback, tb_callback, checkpoint_callback, early_stopping_callback], 
    workers=4,
    use_multiprocessing = True,
)

### Plotting

In [ ]:
#history of training and validation accuracy (only works if model trained and not preloaded)
plt.plot(history.history['accuracy'], label='training acc')
plt.plot(history.history['val_accuracy'], label='validation acc')
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# Save Model

In [ ]:
## SAVE THE MODEL
model.save("saved_models/" + filename)

model.summary()

# Model Evaluation
## Model Prediction

### New Prediction

In [ ]:
## Model Prediction

batch_size = 32

#test prediction storage
y_test_pred = model.predict(
        x_test, 
        batch_size=batch_size,
        workers=4,
        use_multiprocessing = True,
    ).flatten()

#round probabilities to 0 or 1
y_test_pred = np.round(y_test_pred)


with open('predictions/' + 'HW4_predictions_SethCram' + '_' + filename + '.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    
    for i in range(1248):
        if i == 0:
            writeList = ["", "name", "pred"]
        else:
            writeList = [i, "testImgs\\" + str(i-1) + ".png", y_test_pred[i-1]]
        
        writer.writerow(writeList)


### Translate Old Prediction

In [ ]:
filename = 'HW4_predictions_SethCram_ConvNextTiny_firstpass_model73.8%.csv'

#for translating old preds to new format (requires manual insertion of 1st row)
df=pd.read_csv('predictions/' + filename, sep = ',')
print(df)

with open('predictions/' + filename, 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    
    for i in range(1248):
        if i == 0:
            writeList = ["", "name", "pred"]
        else:
            #translate old pred into new format
            writeList = [i, "testImgs\\" + df.values[i-1][0], df.values[i-1][1]]
        
        writer.writerow(writeList)

## Train Accuracy

In [ ]:
#train accuracy
y_train_pred = model.predict(
        x_train, 
        batch_size=batch_size,
        #workers=4,
        #use_multiprocessing = True,
    ).flatten()

#round probabilities to 0 or 1
y_train_pred = np.round(y_train_pred)

train_acc = np.sum(y_train.flatten() == y_train_pred) / y_train.shape[0]

print(f'train accuracy: {train_acc * 100}%')

In [ ]:
loss0, accuracy0 = model.evaluate(x_train, y_train, batch_size=batch_size,)